**Executed:** Sat Mar 18 01:17:42 2017

**Duration:** 4 seconds.

**Autogenerated from:** [./param-learning-template.ipynb](./param-learning-template.ipynb)

In [1]:
# default values of file paths, assuming quantum_fog is working dir
in_bif = 'examples_cbnets/earthquake.bif'
in_dot = 'examples_cbnets/earthquake.dot'
in_csv = 'learning/training_data_c/earthquake.csv'
qfog_path = None

In [2]:
# Cell inserted during automated execution.
in_bif = 'examples_cbnets/WetGrass.bif'
in_dot = 'examples_cbnets/WetGrass.dot'
in_csv = 'learning/training_data_c/WetGrass.csv'
qfog_path = '/home/jupyter/Notebooks/Quantum/quantum-fog'

**Parameter Learning Template**

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Learned-Network-Parameters" data-toc-modified-id="Learned-Network-Parameters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Learned Network Parameters</a></div><div class="lev2 toc-item"><a href="#QFog" data-toc-modified-id="QFog-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>QFog</a></div><div class="lev2 toc-item"><a href="#bnlearn" data-toc-modified-id="bnlearn-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>bnlearn</a></div><div class="lev2 toc-item"><a href="#Summary-of-Running-Times" data-toc-modified-id="Summary-of-Running-Times-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Summary of Running Times</a></div>

In [3]:
import pandas as pd
import numpy as np
from graphviz import Source

import warnings
warnings.filterwarnings("ignore", module="rpy2")

import rpy2
%load_ext rpy2.ipython
%R library("bnlearn");
%R library("Rgraphviz");

In [4]:
import os
import sys
if not qfog_path:
    os.chdir('../../')
else:
    os.chdir(qfog_path)
cwd = os.getcwd()
sys.path.insert(0,cwd)
print("cwd=", cwd)
from learning.NetParamsLner import *

cwd= /home/jupyter/Notebooks/Quantum/quantum-fog


Read in_csv, create Pandas Dataframe with it, push dataframe into R

In [5]:
states_df = pd.read_csv(in_csv)
states_df.head()

,Cloudy,Sprinkler,Rain,WetGrass
0,1,0,1,1
1,1,0,1,1
2,0,0,1,1
3,1,0,1,1
4,0,1,0,1


In [6]:
states_df.tail()

,Cloudy,Sprinkler,Rain,WetGrass
1995,0,1,0,1
1996,1,1,1,1
1997,1,0,0,0
1998,0,1,1,1
1999,1,1,1,1


In [7]:
%Rpush states_df
%R str(states_df)

'data.frame':	2000 obs. of  4 variables:
 $ Cloudy   : int  1 1 0 1 0 1 0 1 1 1 ...
 $ Sprinkler: int  0 0 0 0 1 1 0 0 1 0 ...
 $ Rain     : int  1 1 1 1 0 1 1 0 0 0 ...
 $ WetGrass : int  1 1 1 1 1 1 1 0 1 0 ...


For bnlearn, data.frame columns cannot be int type, must be changed to factor.
data.frame is a list so can use lapply

In [8]:
%R states_df[] <- lapply(states_df, factor)
%R str(states_df)

'data.frame':	2000 obs. of  4 variables:
 $ Cloudy   : Factor w/ 2 levels "0","1": 2 2 1 2 1 2 1 2 2 2 ...
 $ Sprinkler: Factor w/ 2 levels "0","1": 1 1 1 1 2 2 1 1 2 1 ...
 $ Rain     : Factor w/ 2 levels "0","1": 2 2 2 2 1 2 2 1 1 1 ...
 $ WetGrass : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 1 2 1 ...


# Learned Network Parameters

## QFog

In [9]:
# emp = empirical, learned
is_quantum = False
bnet = BayesNet.read_bif(in_bif, is_quantum)
bnet_emp = BayesNet.read_dot(in_dot)
vtx_to_states = bnet.get_vtx_to_state_names()
bnet_emp.import_nd_state_names(vtx_to_states)


lnr = NetParamsLner(is_quantum, bnet_emp, states_df)

In [10]:
%%capture qfog_params_time
%time lnr.learn_all_bnet_pots()

In [11]:
print(qfog_params_time)

CPU times: user 31.1 ms, sys: 0 ns, total: 31.1 ms
Wall time: 31.2 ms



In [12]:
lnr.compare_true_and_emp_pots(bnet, bnet_emp)


node= WetGrass
true:
['Sprinkler', 'Rain', 'WetGrass']
[[[ 0.99  0.01]
  [ 0.01  0.99]]

 [[ 0.01  0.99]
  [ 0.01  0.99]]]
empirical:
['Sprinkler', 'Rain', 'WetGrass']
[[[ 0.98982188  0.01017812]
  [ 0.00613497  0.99386503]]

 [[ 0.00643777  0.99356223]
  [ 0.02147239  0.97852761]]]

node= Cloudy
true:
['Cloudy']
[ 0.5  0.5]
empirical:
['Cloudy']
[ 0.486  0.514]

node= Rain
true:
['Cloudy', 'Rain']
[[ 0.4  0.6]
 [ 0.5  0.5]]
empirical:
['Cloudy', 'Rain']
[[ 0.38168724  0.61831276]
 [ 0.47470817  0.52529183]]

node= Sprinkler
true:
['Cloudy', 'Sprinkler']
[[ 0.2  0.8]
 [ 0.7  0.3]]
empirical:
['Cloudy', 'Sprinkler']
[[ 0.16872428  0.83127572]
 [ 0.69844358  0.30155642]]


## bnlearn


In [13]:
%Rpush in_bif
%R bn.fit = read.bif(in_bif)
%R str(bn.fit)

List of 4
 $ Cloudy   :List of 4
  ..$ node    : chr "Cloudy"
  ..$ parents : chr(0) 
  ..$ children: chr [1:2] "Sprinkler" "Rain"
  ..$ prob    : table [1:2(1d)] 0.5 0.5
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "state0" "state1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Sprinkler:List of 4
  ..$ node    : chr "Sprinkler"
  ..$ parents : chr "Cloudy"
  ..$ children: chr "WetGrass"
  ..$ prob    : table [1:2, 1:2] 0.2 0.8 0.7 0.3
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ Sprinkler: chr [1:2] "state0" "state1"
  .. .. ..$ Cloudy   : chr [1:2] "state0" "state1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ WetGrass :List of 4
  ..$ node    : chr "WetGrass"
  ..$ parents : chr [1:2] "Sprinkler" "Rain"
  ..$ children: chr(0) 
  ..$ prob    : table [1:2, 1:2, 1:2] 0.99 0.01 0.01 0.99 0.01 0.99 0.01 0.99
  .. ..- attr(*, "dimnames")=List of 3
  .. .. ..$ WetGrass : chr [1:2] "state0" "state1"
  .. .. ..$ Sprinkler: chr [1:2] "state0" "state1"
  .. .. ..$ Rai

In [14]:
%R bn = bn.net(bn.fit)
%R str(bn)

List of 3
 $ learning:List of 6
  ..$ whitelist: NULL
  ..$ blacklist: NULL
  ..$ test     : chr "none"
  ..$ ntests   : num 0
  ..$ algo     : chr "empty"
  ..$ args     : list()
 $ nodes   :List of 4
  ..$ Cloudy   :List of 4
  .. ..$ mb      : chr [1:2] "Sprinkler" "Rain"
  .. ..$ nbr     : chr [1:2] "Sprinkler" "Rain"
  .. ..$ parents : chr(0) 
  .. ..$ children: chr [1:2] "Sprinkler" "Rain"
  ..$ Sprinkler:List of 4
  .. ..$ mb      : chr [1:3] "Cloudy" "WetGrass" "Rain"
  .. ..$ nbr     : chr [1:2] "Cloudy" "WetGrass"
  .. ..$ parents : chr "Cloudy"
  .. ..$ children: chr "WetGrass"
  ..$ WetGrass :List of 4
  .. ..$ mb      : chr [1:2] "Sprinkler" "Rain"
  .. ..$ nbr     : chr [1:2] "Sprinkler" "Rain"
  .. ..$ parents : chr [1:2] "Sprinkler" "Rain"
  .. ..$ children: chr(0) 
  ..$ Rain     :List of 4
  .. ..$ mb      : chr [1:3] "Cloudy" "Sprinkler" "WetGrass"
  .. ..$ nbr     : chr [1:2] "Cloudy" "WetGrass"
  .. ..$ parents : chr "Cloudy"
  .. ..$ children: chr "WetGrass"
 $ ar

In [15]:
%%capture bnlearn_params_time
%time %R rfit = bn.fit(bn, data = states_df);

In [16]:
print(bnlearn_params_time)

CPU times: user 8.57 ms, sys: 39 us, total: 8.61 ms
Wall time: 8.73 ms



In [17]:
%R str(rfit)

List of 4
 $ Cloudy   :List of 4
  ..$ node    : chr "Cloudy"
  ..$ parents : chr(0) 
  ..$ children: chr [1:2] "Sprinkler" "Rain"
  ..$ prob    : table [1:2(1d)] 0.486 0.514
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "0" "1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Sprinkler:List of 4
  ..$ node    : chr "Sprinkler"
  ..$ parents : chr "Cloudy"
  ..$ children: chr "WetGrass"
  ..$ prob    : table [1:2, 1:2] 0.169 0.831 0.698 0.302
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ Sprinkler: chr [1:2] "0" "1"
  .. .. ..$ Cloudy   : chr [1:2] "0" "1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ WetGrass :List of 4
  ..$ node    : chr "WetGrass"
  ..$ parents : chr [1:2] "Sprinkler" "Rain"
  ..$ children: chr(0) 
  ..$ prob    : table [1:2, 1:2, 1:2] 0.98982 0.01018 0.00644 0.99356 0.00613 ...
  .. ..- attr(*, "dimnames")=List of 3
  .. .. ..$ WetGrass : chr [1:2] "0" "1"
  .. .. ..$ Sprinkler: chr [1:2] "0" "1"
  .. .. ..$ Rain     : chr [1:2] "0" "1"
  ..- at

In [18]:
%%R
j = 0
for( nd in nodes(bn.fit)){
    j = j + 1
    cat('----------------', nd[1], '\n')
    cat('true:\n')
    print(coef(bn.fit)[j])
    cat('empirical:\n')
    print(coef(rfit)[j])
}

---------------- Cloudy 
true:
$Cloudy
state0 state1 
   0.5    0.5 

empirical:
$Cloudy
    0     1 
0.486 0.514 

---------------- Sprinkler 
true:
$Sprinkler
         Cloudy
Sprinkler state0 state1
   state0    0.2    0.7
   state1    0.8    0.3

empirical:
$Sprinkler
         Cloudy
Sprinkler         0         1
        0 0.1687243 0.6984436
        1 0.8312757 0.3015564

---------------- WetGrass 
true:
$WetGrass
, , Rain = state0

        Sprinkler
WetGrass state0 state1
  state0   0.99   0.01
  state1   0.01   0.99

, , Rain = state1

        Sprinkler
WetGrass state0 state1
  state0   0.01   0.01
  state1   0.99   0.99


empirical:
$WetGrass
, , Rain = 0

        Sprinkler
WetGrass           0           1
       0 0.989821883 0.006437768
       1 0.010178117 0.993562232

, , Rain = 1

        Sprinkler
WetGrass           0           1
       0 0.006134969 0.021472393
       1 0.993865031 0.978527607


---------------- Rain 
true:
$Rain
        Cloudy
Rain     state0 state1
  st

## Summary of Running Times

In [19]:
print("QFog:\n", qfog_params_time)
print("bnlearn:\n", bnlearn_params_time)

QFog:
 CPU times: user 31.1 ms, sys: 0 ns, total: 31.1 ms
Wall time: 31.2 ms

bnlearn:
 CPU times: user 8.57 ms, sys: 39 us, total: 8.61 ms
Wall time: 8.73 ms

